In [ ]:
# Update sklearn to prevent version mismatches
#!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
#!pip install joblib

In [2]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [3]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [4]:
# Set features. This will also be used as your x values.
selected_features = df[["koi_period","koi_srad"]] # what should be my x values?


# Create a Train Test Split

Use `koi_disposition` for the y values

In [17]:
# y-values
y = df["koi_disposition"]
target_names = ["CONFIRMED","FALSE POSITIVE"]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(selected_features, y, random_state = 42)

In [18]:
x_train.head()

,koi_period,koi_srad
6122,6.768901,1.125
6370,0.733726,0.797
2879,7.652707,0.963
107,7.953547,0.779
29,4.959319,1.082


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [19]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

scale = preprocessing.MinMaxScaler()
scaled_x_train = scale.fit_transform(x_train)
scaled_x_test = scale.fit_transform(x_test)

print(scale)
print(scaled_x_train)

# Encode the target
encoder = preprocessing.LabelEncoder()
encoder.fit(y)

y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

print(encoder.classes_)
print(y_train)
print(y_test)

MinMaxScaler(copy=True, feature_range=(0, 1))
[[0.00804828 0.00564746]
 [0.00058597 0.00382426]
 [0.00914108 0.00474698]
 ...
 [0.00728334 0.0153304 ]
 [0.00381261 0.01413532]
 [0.00248788 0.00415221]]
['CANDIDATE' 'CONFIRMED' 'FALSE POSITIVE']
[0 2 2 ... 2 2 2]
[2 0 2 ... 1 1 1]


# Train the Model

In [24]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(scaled_x_train, y_train)
model.fit(scaled_x_test, y_test)

print(f"Training Data Score: {model.score(scaled_x_train, y_train)}")
print(f"Testing Data Score: {model.score(scaled_x_test, y_test)}")

Training Data Score: 0.5056265496852946
Testing Data Score: 0.4879862700228833


/Users/john/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/john/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/john/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/john/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [26]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [27]:
# Train the model with GridSearch
grid.fit(x_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] C=1, gamma=0.0001 ...............................................


/Users/john/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


ValueError: Invalid parameter gamma for estimator LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False). Check the list of available parameters with `estimator.get_params().keys()`.

In [29]:
print(grid2.best_params_)
print(grid2.best_score_)

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'machine_learning_hw_attempt.sav'
joblib.dump(your_model, filename)